In [2]:
from operator import itemgetter
import fitz
import json

In [3]:
def fonts(doc, granularity=False):
    """Extracts fonts and their usage in PDF documents.

    :param doc: PDF document to iterate through
    :type doc: <class 'fitz.fitz.Document'>
    :param granularity: also use 'font', 'flags' and 'color' to discriminate text
    :type granularity: bool

    :rtype: [(font_size, count), (font_size, count}], dict
    :return: most used fonts sorted by count, font style information
    """
    styles = {}
    font_counts = {}

    for page in doc:
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # block contains text
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if granularity:
                            identifier = "{0}_{1}_{2}_{3}".format(s['size'], s['flags'], s['font'], s['color'])
                            styles[identifier] = {'size': s['size'], 'flags': s['flags'], 'font': s['font'],
                                                  'color': s['color']}
                        else:
                            identifier = "{0}".format(s['size'])
                            styles[identifier] = {'size': s['size'], 'font': s['font']}

                        font_counts[identifier] = font_counts.get(identifier, 0) + 1  # count the fonts usage

    font_counts = sorted(font_counts.items(), key=itemgetter(1), reverse=True)

    if len(font_counts) < 1:
        raise ValueError("Zero discriminating fonts found!")

    return font_counts, styles

In [4]:
def font_tags(font_counts, styles):
    """Returns dictionary with font sizes as keys and tags as value.

    :param font_counts: (font_size, count) for all fonts occuring in document
    :type font_counts: list
    :param styles: all styles found in the document
    :type styles: dict

    :rtype: dict
    :return: all element tags based on font-sizes
    """
    p_style = styles[font_counts[0][0]]  # get style for most used font by count (paragraph)
    p_size = p_style['size']  # get the paragraph's size

    # sorting the font sizes high to low, so that we can append the right integer to each tag
    font_sizes = []
    for (font_size, count) in font_counts:
        font_sizes.append(float(font_size))
    font_sizes.sort(reverse=True)

    # aggregating the tags for each font size
    idx = 0
    size_tag = {}
    for size in font_sizes:
        idx += 1
        if size == p_size:
            idx = 0
            size_tag[size] = '<p>'
        if size > p_size:
            size_tag[size] = '<h{0}>'.format(idx)
        elif size < p_size:
            size_tag[size] = '<s{0}>'.format(idx)

    return size_tag

In [8]:
def headers_para(doc, size_tag, name):
    """Scrapes headers & paragraphs from PDF and return texts with element tags.
    :param doc: PDF document to iterate through
    :type doc: <class 'fitz.fitz.Document'>
    :param size_tag: textual element tags for each size
    :type size_tag: dict
    :rtype: list
    :return: texts with pre-prended element tags
    """
    header_para = []  # list with headers and paragraphs
    first = True  # boolean operator for first header
    previous_s = {}  # previous span
    count =0
    for page in doc:
        count+=1
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # this block contains text

                # REMEMBER: multiple fonts and sizes are possible IN one block

                block_string = ""  # text found in block
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if s['text'].strip():  # removing whitespaces:
                            if first:
                                previous_s = s
                                first = False
                                block_string = s['text']
                            else:
                                if s['size'] == previous_s['size']:

                                    if block_string and all((c == "|") for c in block_string):
                                        # block_string only contains pipes
                                        block_string =  s['text']
                                    if block_string == "":
                                        # new block has started, so append size tag
                                        block_string =  s['text']
                                    else:  # in the same block, so concatenate strings
                                        block_string += " " + s['text']

                                else:
                                    header_para.append(block_string)
                                    block_string = s['text']

                                previous_s = s

                    # new block started, indicating with a pipe
                    # =========================================
                    block_string += " "
                    # block_string += "\n"

                
                header_para.append(block_string)
        with open(f"F:/Asha/text-as-page/{name}-page-{count}.txt", 'w', encoding='utf-8') as text_file:
                for para in header_para:
                    text_file.write(para + "\n")
        # with open(f"C:/Users/DELL/Desktop/Asha/text-as-page/test{count}.txt", 'w') as json_out:
        #     json.dump(header_para, json_out)
        header_para.clear()

    return header_para

In [9]:

name = "asha_ncd"
document = "F:/Asha/Asha_pdfs/pdfs/asha_ncd.pdf"
doc = fitz.open(document)
page = doc[0]

font_counts, styles = fonts(doc, granularity=False)

size_tag = font_tags(font_counts, styles)

elements = headers_para(doc, size_tag, name)

In [30]:
# with open("C:/Users/DELL/Desktop/Asha/text-as-page/test.txt", 'w') as json_out:
#     json.dump(elements, json_out)

In [31]:
size_tag

{58.818363189697266: '<h1>',
 48.0: '<h2>',
 36.0: '<h3>',
 34.1525993347168: '<h4>',
 32.25523376464844: '<h5>',
 26.773120880126953: '<h6>',
 26.563133239746094: '<h7>',
 24.6657657623291: '<h8>',
 22.768400192260742: '<h9>',
 20.87103271484375: '<h10>',
 13.281566619873047: '<h11>',
 12.0: '<h12>',
 11.384200096130371: '<h13>',
 11.0: '<p>',
 9.486832618713379: '<s1>',
 8.5381498336792: '<s2>',
 8.0: '<s3>',
 6.0: '<s4>',
 5.0: '<s5>',
 4.0: '<s6>'}

In [51]:
# # # # Clear the folder
import os

folder_path = "C:/Users/DELL/Desktop/Asha/text-as-page"  # Replace with the path to your folder
def clearFolder(folder_path):
    # Iterate over the files in the folder and delete them
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path):
                os.remove(file_path)
        except Exception as e:
            print(f"Error deleting {file_path}: {e}")

    print("All files in the folder have been deleted.")

In [52]:
clearFolder(folder_path)

All files in the folder have been deleted.
